# Proyecto Final Área de Imágenes
### EL5206 Laboratorio de Inteligencia Computacional y Robótica

Integrantes:
* José Díaz
* Luis Jiménez

El proyecto a realizar consiste en el desarrollo e implementación de un
algoritmo de búsqueda de imágenes del tipo CBIR, el cual hará comparaciones entre
una imagen de consulta e imágenes en la base de datos INRIA Holidays dataset a través de sus vectores de características.
Para esta implementación se utilizarán dos extractores: uno clásico (HOG) y otro
basado en redes convolucionales (CNN).

Nota: con el fin de que el código funcione, todas las imágenes de la base de datos se almacenan
en la carpeta JPG, que está en el mismo directorio que este notebook Jupyter.

### Librerías útiles

Carga de librerías útiles para el desarrollo del proyecto.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from PIL import Image
from torchvision import transforms
import os
from tqdm.notebook import tqdm
from skimage.feature import hog
from skimage import data, exposure

### 1. Extracción de características

Para la extracción de carecterísticas se utilizará la función FEATURE_EXTRACTOR,
que extrae el vector de características de una imagen, utilizando el extractor
de tipo HOG  (clásico) o tipo CNN (red neuronal).

Los parámetros que se le entregan a la función son el directorio de
la imagen o imágenes (PATH_LIST), un directorio de guardado (SAVE_PATH)
y el tipo de extractor (EXTRACTOR_TYPE). Cada vector de características
asociado a una imagen se guarda en el directorio de guardado en forma de un
archivo binario de formato .npy, el cual puede ser recuperado como array de Numpy
mediante la función np.load.

Hay que precisar que los directorios donde se guardarán los vectores de
características deben estar ya creados para que el código funcione, por lo
que en el mismo directorio de este notebook se requieren 2 carpetas: JPG_HOG para
los vectores extraídos con el método HOG, y JPG_CNN para la extracción
con CNN.

In [2]:
def feature_extractor(path_list, save_path, extractor_type):

    if type(path_list) != type([]):
        path_list = [path_list]

    if extractor_type == 'HOG':

        for file in tqdm(path_list):
            image = Image.open(file)
            image = image.resize((224,224))

            hog_vect = hog(image, orientations=16, pixels_per_cell=(8, 8),
                            cells_per_block=(1, 1), feature_vector=True, multichannel=True)

            vect_name = vect_name = os.path.join(save_path, 'hog_' + os.path.basename(file[:-4]))
            #print(type(hog_vect))
            #print(hog_vect.shape)
            np.save(vect_name, hog_vect)


    elif extractor_type == 'CNN':

        model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', pretrained=True)
        model = torch.nn.Sequential(*(list(model.children())[:-1]))

        preprocess = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
         # create a mini-batch as expected by the model


        # move the input and model to GPU for speed if available
        if torch.cuda.is_available():
            input_batch = input_batch.to('cuda')
            model.to('cuda')

        with torch.no_grad():
            for file in tqdm(path_list):
                image = Image.open(file)
                image = image.resize((224,224))
                input_tensor = preprocess(image)
                input_batch = input_tensor.unsqueeze(0)

                output = model(input_batch)
                output = output.numpy().ravel()
                vect_name = os.path.join(save_path, 'cnn_' + os.path.basename(file[:-4]))
                #print(type(hog_vect))
                #print(hog_vect.shape)
                np.save(vect_name, output)

    else:
        print('Unknown extractor, please use either "HOB" or "CNN"')

#### Lista de directorios de todas las imágenes de la BBDD

In [5]:
file_paths = [os.path.join('jpg', file) for file in os.listdir('jpg')]

#### Extracción usando HOG

In [6]:
feature_extractor(file_paths, 'jpg_hog/', 'HOG')

#### Extracción usando CNN

In [4]:
feature_extractor(file_paths, 'jpg_cnn/', 'CNN')

Using cache found in C:\Users\Luis Jiménez/.cache\torch\hub\pytorch_vision_v0.6.0


ImportError: cannot import name 'resnext50_32x4d' from 'torchvision.models.resnet' (c:\users\luis jiménez\desktop\importante\u\lab_inteligencia\proyecto\proyecto_final_el5206\venv\lib\site-packages\torchvision\models\resnet.py)

### 2. Métricas

Se definen métricas de comparación entre vectores de
características de imágenes, correspondientes a distancia entre
vectores.

#### Distancia euclideana

In [4]:
def dist_euc(X, Y):

    dist = np.linalg.norm(X-Y)
    return dist

#### Distancia chi-cuadrado [1]

In [5]:
def dist_chi2(X, Y):

    chi = 0.5 * np.sum([((x - y) ** 2) / (x + y)
                      for (x, y) in zip(X, Y)])
    return chi

##### Pruebas de dist_euc y dist_chi2

In [6]:
X = np.array([1,2,3])
Y = np.array([5,6,7])
print(dist_euc(X, Y), dist_chi2(X, Y))

6.928203230275509 3.133333333333333


#### Función para elegir tipo de distancia

In [7]:
def dist_vectors(X, Y, dist_type):

    if dist_type == 'euc':

        dist = dist_euc(X, Y)

    elif dist_type == 'chi2':

        dist = dist_chi2(X, Y)

    return dist

### 3. Comparación de una imagen de consulta con la BBDD

En esta sección, para cada imagen de consulta, se calcula la distancia entre
su vector de características con todos los vectores de características de
la base de datos.

1 + CLASE (3d) + ID_imagen (2d)

RECORDATORIO: ID_imagen = 00 -> CONSULTA

DISTINTO DE 00 ES BBDD

#### Separación entre grupo de consulta y grupo de BBDD

In [11]:
w_path = 'jpg_hog/'
img_query = []
query_names = []
img_db = []
db_names = []

for file in tqdm(os.listdir(w_path)):

    vector_path = w_path + file
    feat_vector = np.load(vector_path)
    ID_image = vector_path[-6:-4]

    if ID_image == '00':
        img_query.append(feat_vector)
        query_names.append(file[-10:-4])

    else:
        img_db.append(feat_vector)
        db_names.append(file[-10:-4])

#### Cálculo de distancias entre imagen de consulta e imágenes de la BBDD

Sintaxis:
*   distances_euc{'nombre_img_consulta'} = vector_distancias

In [19]:
#uso distancia euclideana
distances_euc = {}
for i in range(len(img_query)):

    query_vect = img_query[i]
    query_name = query_names[i]
    distances = []

    for j in range(len(img_db)):

        distances.append(dist_vectors(query_vect, img_db[j],
                                      dist_type = 'euc'))

    distances_euc[query_name] = np.array(distances)


### 4. Ordenamiento de imágenes por relevancia (medida de Ranking)

A continuación, para cada imagen de consulta, se ordenan de menor a mayor
las distancias obtenidas al hacer la comparación con la base de datos.
Adicionalmente, se conservan los índices de las distancias previo al
ordenamiento, ya que dichos índices corresponden a las imágenes de la
base de datos, que se necesitarán para la medida de ranking.

Sintaxis:
*   dist_sort_euc{'nombre_img_consulta'} = vector_distancias
*   dist_sort_euc_index{'nombre_img_consulta'} = pos_previas_dist

para cada imagen "query",
calculaba sus distancias con todas las imagenes en la
base de datos, las ordenaba, y calculaba el ranking
sacando el promedio de la posición en que quedaron
las imágenes de la misma clase a la query

In [22]:
#uso distancia euclideana
dist_sort_euc = {}
dist_sort_euc_index = {}
img_query_names = list(distances_euc.keys())

for i in range(len(distances_euc)):

    dist_vector = distances_euc[img_query_names[i]]
    dist_vector_index = np.argsort(dist_vector)
    dist_vector_sort = np.sort(dist_vector)
    dist_sort_euc[img_query_names[i]] = dist_vector_sort
    dist_sort_euc_index[img_query_names[i]] = dist_vector_index

Luego, con las imágenes ordenadas, se calcula una medida de ranking,
que entrega la posición promedio de las imágenes que entrega el buscador
al realizar la consulta de una imagen.

Las imágenes consideradas relevantes para la consulta de una imagen particular
corresponden a imágenes de la base de datos que pertenecen a la misma
clase que la imagen de consulta.

La función que se muestra a continuación calcula el ranking para
una sola imagen de consulta, además del número de imágenes relevantes
para la consulta en curso.

In [26]:
def rank(img_query_name, rel_pos = False):

    Nrel = 0
    sum = 0
    img_query_class = img_query_name[1:2]
    img_db_pos = dist_sort_euc_index[img_query_name]
    relevant_pos = []

    for pos in img_db_pos:

        img_db_class = db_names[pos][1:2]

        if img_db_class == img_query_class:

            sum += pos
            Nrel += 1
            relevant_pos.append(pos)

    r = sum/Nrel
    r_pos = np.array(relevant_pos)

    if rel_pos == True:
        return r, Nrel, r_pos

    else:
        return r, Nrel

### 5. Comparación de métodos de extracción

Se necesita evaluar la robustez de los métodos de extracción utilizados
en este proyecto, y una forma de compararlos es mediante la normalización
de la medida de Ranking que se utilizó anteriormente.

In [27]:
def rank_norm(img_query_name):

    r, Nrel = rank(img_query_name)
    N = len(db_names)

    r_norm = (r*Nrel - (Nrel(Nrel + 1))/2) * (N*Nrel)**(-1)

    return r_norm

### 6. Ejemplos de uso del algoritmo

A continuación se muestran algunos ejemplos de consulta de imágenes
y la respuesta del método, cambiando tipos de extractores y métricas
de distancias utilizadas.

### 7. Optimización algorítmica del método

### Referencias utilizadas

[1] https://www.geeksforgeeks.org/chi-square-distance-in-python/

